# Descriptive Statistics
### base on Chua, Alton Y.K. and Banerjee, Snehasish 2016
#### http://www.sciencedirect.com/science/article/pii/S074756321530131X

In [ ]:
import sys
sys.path.append("../pylinguistics/pylinguistics/")

import Pylinguistics as pl
import pandas as pd
import plotsfunc as pf
import numpy as np
%matplotlib inline

reviews_features = pd.read_csv('reviews.csv.gz', compression='gzip')
#reviews_features = pd.read_csv('amazon-help.csv.gz')

In [2]:
## adiciona coluna de helpfulness no corpus
def helpf(x): 
    try:
        return float ( float(x['thumbsup']) / (float(x['thumbsup']) + float(x['thumbsdown'])) )
    except:
        return 0
    
def thumbssum(x): 
    try:
        return int(x['thumbsup']) + int(x['thumbsdown']) 
    except:
        return 0
    
def fillnanup(x): 
    try:
        return float(x['thumbsup'])
    except:
        return 0
    
def fillnandown(x): 
    try:
        return float(x['thumbsdown'])
    except:
        return 0
    
def length(x): 
    try:
        return int(x['word_count']) * int(x['mean_word_length'])
    except:
        return 0

In [12]:
reviews_features['thumbsup'] = reviews_features.apply(fillnanup,axis=1)
reviews_features['thumbsdown'] = reviews_features.apply(fillnandown,axis=1)
reviews_features['helpfulness'] = reviews_features.apply(helpf,axis=1)
reviews_features['thumbstotal'] = reviews_features.apply(thumbssum,axis=1)
reviews_features['length'] = reviews_features.apply(length,axis=1)
reviews_features = reviews_features[reviews_features.thumbstotal>4]

## para usar no weka
#reviews_features.to_csv('amazon-help.csv',encoding='utf-8')

In [4]:
def mean_data(df):
    mean_stats = []
    mean_stats.append(df.length.mean())
    mean_stats.append(df.redability.mean())
    mean_stats.append(df.LexicalDiversty.mean())
    mean_stats.append(df.verbIncidence.mean())
    mean_stats.append(df.adpPronRatio.std())
    mean_stats.append(len(df))
    return mean_stats

def std_data(df):
    std_stats = []
    std_stats.append(df.word_count.std())
    std_stats.append(df.redability.std())
    std_stats.append(df.LexicalDiversty.std())
    std_stats.append(df.verbIncidence.std())
    std_stats.append(df.adpPronRatio.std())
    std_stats.append(len(df))
    return std_stats

columns = ['Length','Redability','Verbs incidence','Adv Pron Ratio','Lexical Diversty','Reviews']

In [13]:
# split mixed subset
mixed_reviews = reviews_features[reviews_features.stars > 1]
mixed_reviews = mixed_reviews[reviews_features.stars < 5]

In [14]:
# split favorable subset
favorable_reviews = reviews_features[reviews_features.stars == 5]

In [15]:
# split unfavorable subset
unfavorable_reviews = reviews_features[reviews_features.stars == 1]

In [43]:
stats = pd.DataFrame(data={'mean_mixed': np.array(mean_data(mixed_reviews))}, index=columns)
stats['std_mixed'] = std_data(mixed_reviews)
stats['mean_favorable'] = mean_data(favorable_reviews)
stats['std_favorable'] = std_data(favorable_reviews)
stats['mean_unfavorable'] = mean_data(unfavorable_reviews)
stats['std_unfavorable'] = std_data(unfavorable_reviews)

## Descriptive Table Buscape Reviews
### statistics (mean ± sd) of reviews as a function of review sentiment

In [36]:
stats

,mean_mixed,std_mixed,mean_favorable,std_favorable,mean_unfavorable,std_unfavorable
Length,86.346112,70.885989,59.604149,57.262348,97.819423,69.541724
Redability,15.725786,6.439299,15.328631,7.127779,15.756449,9.207110
Verbs incidence,0.723967,0.106067,0.774523,0.139167,0.688236,0.127435
Adv Pron Ratio,177.652980,54.690528,176.992329,78.050514,198.502380,60.183015
Lexical Diversty,1.213923,1.213923,1.530900,1.530900,1.071330,1.071330
Reviews,6391.000000,6391.000000,2410.000000,2410.000000,659.000000,659.000000


## Descriptive Table Amazon Reviews

In [44]:
stats

,mean_mixed,std_mixed,mean_favorable,std_favorable,mean_unfavorable,std_unfavorable
Length,180.697476,193.366813,159.306030,206.239197,115.249932,129.536028
Redability,8.467965,6.872498,7.778845,7.251468,7.291651,5.495074
Verbs incidence,0.684189,0.131205,0.712311,0.139042,0.739150,0.125694
Adv Pron Ratio,190.901345,39.462693,185.130498,44.608704,204.107767,45.863883
Lexical Diversty,0.552248,0.552248,0.647407,0.647407,0.627895,0.627895
Reviews,6340.000000,6340.000000,5091.000000,5091.000000,3689.000000,3689.000000


## Results of the multiple regression analyses

In [8]:
from scipy.stats import spearmanr
from sklearn.svm import SVR, LinearSVR
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer

def simple_spearman(x,y): return np.abs(spearmanr(x,y)[0])
spearmanr_scorer = make_scorer(simple_spearman)

In [19]:
# create new dict for correlation
mixed_list = []
favorable_list = []
unfavorable_list = []

#svm model
model = LinearSVR(epsilon=0.01)

# build columns list
df_columns = mixed_reviews.select_dtypes(include=['float64','int']).columns.drop('helpfulness')
df_columns

Index([u'ConnectiveAdditiveIncidence', u'redability', u'mean_sentence_length',
       u'percentile_75_sentence_length', u'percentile_75_word_length',
       u'ConnectiveLogicIncidence', u'percentile_90_word_length',
       u'mean_word_length', u'syllable_count', u'avg_word_per_sentence',
       u'LogicIfIncidence', u'percentile_50_sentence_length',
       u'median_sentence_length', u'LogicAndIncidence',
       u'percentile_90_sentence_length', u'median_word_length',
       u'pronIncidence', u'LogicOperatorsIncidence',
       u'percentile_25_word_length', u'ContentDiversty', u'verbIncidence',
       u'functionalIncidence', u'adjectiveIncidence', u'ConnectiveIncidence',
       u'contentIncidence', u'LogicNegationIncidence',
       u'avg_syllables_per_word', u'ConnectiveTemporalIncidence',
       u'sentence_count', u'ConnectiveCasualIncidence', u'advIncidence',
       u'LexicalDiversty', u'adpPronRatio', u'thumbsup', u'thumbsdown',
       u'stars'],
      dtype='object')

In [ ]:
#compute correlation
for i in df_columns:
    #compute mixed score
    labels = mixed_reviews["helpfulness"].values
    features = mixed_reviews[list([i])].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    mixed_list.append(scores.mean())
    #compute favorable score
    labels = favorable_reviews["helpfulness"].values
    features = favorable_reviews[list([i])].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    favorable_list.append(scores.mean())
    #compute unfavorable score
    labels = unfavorable_reviews["helpfulness"].values
    features = unfavorable_reviews[list([i])].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    unfavorable_list.append(scores.mean())
    #print status
    print i

corr_df = pd.DataFrame(data={'mixed': np.array(mixed_list)}, index=df_columns)
corr_df['favorable'] = favorable_list
corr_df['unfavorable'] = unfavorable_list

In [10]:
corr_df

,mixed,favorable,unfavorable
ConnectiveAdditiveIncidence,0.037210,0.054535,NaN
redability,0.032110,0.036676,0.104063
mean_sentence_length,0.037131,0.065071,0.086108
percentile_75_sentence_length,0.069619,0.109616,0.136307
percentile_75_word_length,0.069751,0.018212,0.092850
ConnectiveLogicIncidence,NaN,NaN,NaN
percentile_90_word_length,0.076749,0.067088,0.130757
mean_word_length,0.031962,0.046607,0.079057
syllable_count,0.264287,0.307581,0.309379
avg_word_per_sentence,0.030196,0.050397,0.080424
